# Point Source to Far Field Reflector — One-Shot Benchmark

**`Kernel → Restart & Run All`** — runs the full pipeline without any manual steps:

| Step | What happens |
|------|--------------|
| 1 | **Config** — sizes, benchmark case |
| 2 | **Dependencies** — installs numpy + matplotlib into the live kernel |
| 3 | **Generate** point clouds (quasi-Monte Carlo on sphere) |
| 4 | **Compile** C++ via `make` |
| 5 | **Run** the benchmark |
| 6 | **Visualize** — runs in a subprocess so fresh installs are picked up |

## Step 1 — Configuration

In [ ]:
# ── Point-cloud sizes ─────────────────────────────────────────────────────────
# NK complexity is O(NK²):  1600 → fast (~seconds)  |  16488 → full (~minutes)
NK       = 1600
NK_small = 200     # warm-start sampler  (must be < NK)

# ── Benchmark test-case ───────────────────────────────────────────────────────
# 'test_3D_SquareToCircle_logcost_MonteCarlo.h'
# 'test_3D_SquareToTwoGaussSide_logcost_MonteCarlo.h'
BENCHMARK = 'test_3D_SquareToCircle_logcost_MonteCarlo.h'

# ── Paths ─────────────────────────────────────────────────────────────────────
import os, sys
REPO_ROOT = os.path.abspath('')          # directory this notebook lives in
CODE_DIR  = os.path.join(REPO_ROOT, 'BenchmarkCode')

print(f'Python    : {sys.executable}')
print(f'Repo root : {REPO_ROOT}')
print(f'Code dir  : {CODE_DIR}')
print(f'NK={NK}, NK_small={NK_small}')
print(f'Benchmark : {BENCHMARK}')

## Step 2 — Install / fix dependencies

Uses `sys.executable` so pip targets **this exact kernel's** site-packages.  
Matplotlib is force-reinstalled so its C extension matches the numpy version.  
Visualization runs in a **subprocess** later, so the fresh binaries are loaded cleanly.

In [ ]:
import subprocess, sys

def pip(*args):
    """Run pip via the live kernel's interpreter and stream output."""
    cmd = [sys.executable, '-m', 'pip', *args]
    result = subprocess.run(cmd, capture_output=True, text=True)
    # Show only warnings/errors, not the full install log
    for line in (result.stdout + result.stderr).splitlines():
        if any(k in line for k in ('ERROR', 'error', 'WARNING', 'Successfully', 'already')):
            print(line)
    return result.returncode

# Install numpy first so matplotlib's C extension links against the right version
print('Installing numpy...')
pip('install', 'numpy>=1.21,<2.0', '--upgrade', '--quiet')

# Force-reinstall matplotlib so ft2font.so is recompiled / matched to numpy
print('Installing matplotlib (force-reinstall to fix binary compatibility)...')
pip('install', 'matplotlib', '--upgrade', '--force-reinstall', '--quiet')

# Verify
r = subprocess.run(
    [sys.executable, '-c', 'import numpy, matplotlib; print("numpy", numpy.__version__, "| matplotlib", matplotlib.__version__)'],
    capture_output=True, text=True
)
print(r.stdout.strip() or r.stderr.strip())
print('✓ Dependencies ready')

## Step 3 — Generate Point Clouds

In [ ]:
import math

# ── Halton quasi-random sequence ──────────────────────────────────────────────
def halton(index, base):
    result, f, i = 0.0, 1.0, index
    while i > 0:
        f /= base; result += f * (i % base); i //= base
    return result

# ── Inverse stereographic projection → unit sphere ────────────────────────────
# Matches the C++ formula in Generic_3D_logcost_MonteCarlo.h exactly
def sphere_pt(X, Y, upper=True):
    N2 = X*X + Y*Y; d = 1.0 + N2; z = 1.0 if upper else -1.0
    return (2*X/d, 2*Y/d, z*(1-N2)/d)

def gen_pts(n, upper, skip=0, half=0.6):
    pts, idx = [], skip
    while len(pts) < n:
        pts.append(sphere_pt((halton(idx,2)-0.5)*2*half, (halton(idx,3)-0.5)*2*half, upper))
        idx += 1
    return pts

FMT = lambda v: f'{v:.21e}'
row = lambda pt: ', '.join(FMT(v) for v in pt) + ', '

def write_file(path, lines):
    with open(path, 'w', newline='') as f:
        f.write('\r\n'.join(lines) + '\r\n')

# Generate all clouds
print('Generating points...', end=' ', flush=True)
x_pts  = gen_pts(NK,       upper=True,  skip=0)
y_pts  = gen_pts(NK,       upper=False, skip=0)
xs_pts = gen_pts(NK_small, upper=True,  skip=NK)
ys_pts = gen_pts(NK_small, upper=False, skip=NK)
print('done.')

# MonteCarlo_Pointcloud_3D_128.h
write_file(os.path.join(CODE_DIR, 'QuasiMonteCarlo', 'MonteCarlo_Pointcloud_3D_128.h'), [
    f'#ifndef MonteCarlo_Pointcloud_3D_{NK}', f'#define MonteCarlo_Pointcloud_3D_{NK}', '',
    f'const int NK={NK};', 'const int dim=3;', '',
    'double x[NK][dim]=', '{', *[row(p) for p in x_pts], '};', '', '',
    'double y[NK][dim]=', '{', *[row(p) for p in y_pts], '};', '', '',
    f'const int NK_small={NK_small};', '', '',
    'double x_small[NK_small][dim]=', '{', *[row(p) for p in xs_pts], '};', '', '',
    'double y_small[NK_small][dim]=', '{', *[row(p) for p in ys_pts], '};', '', '',
    '#endif',
])

# 3D_MonteCarlo_Pointcloud_small.h
write_file(os.path.join(CODE_DIR, 'SmallGrid', '3D_MonteCarlo_Pointcloud_small.h'), [
    f'const int NK_small={NK_small};', '',
    '//monte-carlo cloud for both x and y. where reflector cost is applied, last values of y should change sign.',
    '', '',
    'double x_small[NK_small][3]=', '{ ', *[row(p) for p in xs_pts], '};',
    '', '', '', '', '', '', '',
    'double y_small[NK_small][3]=', '{ ', *[row(p) for p in ys_pts], '};',
    '', '',
])

# PushForward_Cloud_128.h
write_file(os.path.join(CODE_DIR, 'PushForward', 'PushForward_Cloud_128.h'), [
    '#ifndef PushForward_Cloud_128', '#define PushForward_Cloud_128', '',
    f'const int Push_Cloud_Size={NK};', '',
    'double Push_Cloud[Push_Cloud_Size][3]=', '{', *[row(p) for p in x_pts], '};',
    '', '', '#endif',
])

print(f'✓ Point clouds written: NK={NK}, NK_small={NK_small}')

## Step 4 — Compile

In [ ]:
import re

# Patch the #include in main.cpp to the chosen benchmark
main_cpp = os.path.join(CODE_DIR, 'main.cpp')
src = open(main_cpp).read()
patched = re.sub(
    r'#include\s+"Benchmarks/[^"]+"',
    f'#include "Benchmarks/{BENCHMARK}"',
    src, count=1
)
if patched != src:
    open(main_cpp, 'w').write(patched)
    print(f'Patched main.cpp → {BENCHMARK}')
else:
    print(f'main.cpp already set to {BENCHMARK}')

print('\nCompiling...')
result = subprocess.run(['make', '-C', CODE_DIR], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print('STDERR:', result.stderr)
    raise RuntimeError(f'Compilation failed (exit {result.returncode})')
print('✓ Compiled successfully')

## Step 5 — Run Benchmark

In [ ]:
import time

print(f'Running benchmark (NK={NK})...')
t0 = time.time()

result = subprocess.run(
    [os.path.join(CODE_DIR, 'main')],
    cwd=CODE_DIR, capture_output=True, text=True
)
elapsed = time.time() - t0

out = result.stdout
print(out[-4000:] if len(out) > 4000 else out)
if result.returncode != 0:
    print('STDERR:', result.stderr[-2000:])
    raise RuntimeError(f'Benchmark failed (exit {result.returncode})')

print(f'✓ Completed in {elapsed:.1f}s')

## Step 6 — Visualize

> **Why subprocess?**  
> The Intel oneAPI image ships a matplotlib compiled against an older numpy.
> Even after `pip install`, the kernel's already-loaded C extension (`.so`) is stale.  
> Running the viz in a **fresh subprocess** guarantees the updated binaries are used,  
> then the saved PNGs are displayed back here via `IPython.display`.

In [ ]:
import glob

def find_output(code_dir):
    patterns = [
        os.path.join(code_dir, 'Results_*', '**', 'Output_*'),
        os.path.join(code_dir, 'Results_*', '*'),
        os.path.join(code_dir, 'Output_*'),
    ]
    dirs = []
    for p in patterns:
        dirs += [d for d in glob.glob(p, recursive=True) if os.path.isdir(d)]
    return max(dirs, key=os.path.getmtime) if dirs else None

output_dir = find_output(CODE_DIR)
if output_dir:
    print(f'Output directory : {output_dir}')
    txts = sorted(f for f in os.listdir(output_dir) if f.endswith('.txt'))
    print(f'Files ({len(txts)}): {txts}')
else:
    raise RuntimeError('No output directory found — did Step 5 succeed?')

In [ ]:
from IPython.display import Image, display

# Run visualize.py in a fresh subprocess so the force-reinstalled matplotlib
# (Step 2) is loaded from scratch.  Errors will show file + line number in
# visualize.py, making them easy to track down.
print('Running visualization subprocess...')
result = subprocess.run(
    [sys.executable, os.path.join(REPO_ROOT, 'visualize.py'),
     '--output-dir', output_dir,
     '--benchmark',  BENCHMARK,
     '--nk',         str(NK)],
    capture_output=True, text=True
)
print(result.stdout.strip())
if result.returncode != 0:
    print('STDERR:', result.stderr[-3000:])
    raise RuntimeError('Visualization subprocess failed')

# Display saved PNGs inline
pngs = [
    os.path.join(output_dir, 'visualization.png'),
    os.path.join(output_dir, 'reflector_analysis.png'),
    os.path.join(output_dir, 'density_comparison.png'),
]
for png in pngs:
    if os.path.exists(png):
        print(f'\n── {os.path.basename(png)} ──')
        display(Image(png))

## Step 7 — Interactive 3D Optical System

Plotly figure showing all three physical objects in a shared 3D space.
Drag to rotate, scroll to zoom, click legend items to toggle traces.

| Colour | Object |
|--------|--------|
| Gold | Source light — upper hemisphere |
| Viridis | Reflector surface (coloured by radius) |
| Steel blue | Target distribution — lower hemisphere (original) |
| Tomato | Pushed / output light — lower hemisphere (reflected) |

In [ ]:
import subprocess, sys, os, importlib, site
import numpy as np

# ── Install plotly if missing ─────────────────────────────────────────────────
try:
    import plotly.graph_objects as go
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'plotly', '--quiet'], check=True)
    user_site = site.getusersitepackages()
    if user_site not in sys.path:
        sys.path.insert(0, user_site)
    importlib.invalidate_caches()
    import plotly.graph_objects as go

# ── scipy for clean Delaunay triangulation of reflector surface ───────────────
# Catch ValueError too: the Intel oneAPI scipy is compiled against an older
# numpy; after our force-reinstall it raises a binary-incompatibility error.
try:
    from scipy.spatial import Delaunay as _Delaunay
    _HAS_SCIPY = True
except (ImportError, ValueError):
    _HAS_SCIPY = False   # falls back to plotly alphahull

# ── Data loaders ──────────────────────────────────────────────────────────────
def _load_vec(path):
    """Header-skipping loader: first line skipped, rest are float rows."""
    if not os.path.exists(path): return None
    with open(path) as fh:
        fh.readline()
        rows = [[float(v) for v in ln.split()] for ln in fh if ln.strip()]
    return np.array(rows) if rows else None

def _load_pts(path):
    """No-header loader: rows with >= 2 floats."""
    if not os.path.exists(path): return None
    rows = [[float(v) for v in ln.split()]
            for ln in open(path) if ln.strip() and len(ln.split()) >= 2]
    return np.array(rows) if rows else None

def _proj_to_sphere_lower(pts2d):
    """Inverse stereographic projection (north pole) → lower hemisphere.
    C++ projects lower-hemisphere y via X=y[0]/(1−y[2]); inverse:
        N²=X²+Y², y=(2X/d, 2Y/d, −(1−N²)/d)  where d=1+N²
    """
    X, Y = pts2d[:, 0], pts2d[:, 1]
    N2 = X**2 + Y**2; d = 1.0 + N2
    return np.column_stack([2*X/d, 2*Y/d, -(1.0 - N2)/d])

# ── Load data ─────────────────────────────────────────────────────────────────
_j = lambda name: os.path.join(output_dir, name)

x_src  = _load_vec(_j("x_MY.txt"))         # source — upper hemisphere (3-D)
p_vals = _load_vec(_j("p_MY.txt"))          # source density P(x)
ref    = _load_vec(_j("Ref_MY.txt"))        # reflector surface (3-D)
r_data = _load_vec(_j("R_MY.txt"))         # reflector radius per point

_yp2d     = _load_pts(_j("Y_Pushed_projected.txt"))
y_push_2d = _yp2d if _yp2d is not None else _load_pts(_j("Y_Pushed_projected_owndisc.txt"))
y_push_3d = _proj_to_sphere_lower(y_push_2d[:, :2]) if y_push_2d is not None else None

R_flat = r_data.flatten() if r_data is not None else None

MAX_PTS = 6_000   # points per trace — keep browser responsive

# ── Build figure ──────────────────────────────────────────────────────────────
fig = go.Figure()

# 1 ── Source hemisphere: scatter colored by P(x) intensity (yellow→red)
if x_src is not None:
    p_flat   = p_vals.flatten() if p_vals is not None else np.ones(len(x_src))
    n_common = min(len(x_src), len(p_flat))
    idx      = np.random.choice(n_common, min(n_common, MAX_PTS), replace=False)
    fig.add_trace(go.Scatter3d(
        x=x_src[idx, 0], y=x_src[idx, 1], z=x_src[idx, 2],
        mode='markers',
        marker=dict(
            size=2.5,
            color=p_flat[idx],
            colorscale='YlOrRd',
            opacity=0.65,
            showscale=False,
        ),
        name='Source  P(x)',
        hovertemplate=(
            'x=%{x:.3f}  y=%{y:.3f}  z=%{z:.3f}'
            '<br>P(x)=%{marker.color:.4f}<extra></extra>'
        ),
    ))

# 2 ── Reflector: physical triangle-mesh surface colored by radius
if ref is not None and len(ref) >= 4:
    r_int = R_flat if (R_flat is not None and len(R_flat) == len(ref)) else ref[:, 2]

    if _HAS_SCIPY:
        # Delaunay triangulation projected onto XY — correct for a 2.5-D surface
        tri     = _Delaunay(ref[:, :2])
        mesh_kw = dict(i=tri.simplices[:, 0],
                       j=tri.simplices[:, 1],
                       k=tri.simplices[:, 2])
        print("Reflector mesh: Delaunay triangulation (scipy)")
    else:
        # Fallback: plotly alpha-hull
        mesh_kw = dict(alphahull=5)
        print("Reflector mesh: alphahull fallback (scipy incompatible with numpy 2.x)")

    fig.add_trace(go.Mesh3d(
        x=ref[:, 0], y=ref[:, 1], z=ref[:, 2],
        **mesh_kw,
        intensity=r_int,
        colorscale='Viridis',
        opacity=0.90,
        showscale=True,
        colorbar=dict(title='Radius', x=1.02, len=0.55, y=0.5, thickness=15),
        name='Reflector surface',
        lighting=dict(ambient=0.5, diffuse=0.8, specular=0.4, roughness=0.4,
                      fresnel=0.2),
        lightposition=dict(x=100, y=200, z=300),
        hovertemplate=(
            'x=%{x:.3f}  y=%{y:.3f}  z=%{z:.3f}'
            '<br>Radius=%{intensity:.4f}<extra></extra>'
        ),
    ))

# 3 ── Output light: single lower-hemisphere surface colored by Q(y) intensity
if y_push_3d is not None and y_push_2d is not None:
    q_color = (y_push_2d[:, 2] if y_push_2d.shape[1] >= 3
               else np.ones(len(y_push_3d)))
    idx = np.random.choice(len(y_push_3d), min(len(y_push_3d), MAX_PTS), replace=False)
    fig.add_trace(go.Scatter3d(
        x=y_push_3d[idx, 0], y=y_push_3d[idx, 1], z=y_push_3d[idx, 2],
        mode='markers',
        marker=dict(
            size=2.5,
            color=q_color[idx],
            colorscale='Blues',
            opacity=0.65,
            showscale=False,
        ),
        name='Output light  Q(y)',
        hovertemplate=(
            'x=%{x:.3f}  y=%{y:.3f}  z=%{z:.3f}'
            '<br>Q(y)=%{marker.color:.4f}<extra></extra>'
        ),
    ))

bench_label = BENCHMARK.replace(".h", "")
fig.update_layout(
    title=dict(
        text=(f'<b>Optical System — {bench_label}</b><br>'
              f'<sup>NK={NK}  ·  '
              f'<span style="color:#e05">■</span> source P(x)  ·  '
              f'<span style="color:#4a8">■</span> reflector (radius)  ·  '
              f'<span style="color:#48f">■</span> output Q(y)  ·  '
              f'drag=rotate  scroll=zoom  hover=values</sup>'),
        x=0.5, xanchor='center',
    ),
    scene=dict(
        xaxis=dict(title='X', showgrid=True),
        yaxis=dict(title='Y', showgrid=True),
        zaxis=dict(title='Z', showgrid=True),
        aspectmode='data',
        camera=dict(eye=dict(x=1.4, y=1.4, z=0.8)),
    ),
    legend=dict(itemsizing='constant', x=0, y=1),
    margin=dict(l=0, r=80, t=100, b=0),
    width=960, height=720,
)

fig.show()
print("Interactive 3D plot ready.")